## ADS509 Module4- Mohammad Mahmoudighaznavi

## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard

In [41]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict



from string import punctuation
from nltk.corpus import stopwords
import re
 

In [42]:
# Some punctuation variations
punctuation = set(punctuation) # speeds up comparison
tw_punct = punctuation - {"#"}

# Stopwords
sw = stopwords.words("english")

# Two useful regex
whitespace_pattern = re.compile(r"\s+")
hashtag_pattern = re.compile(r"^#[0-9a-zA-Z]+")


# and now our functions
def descriptive_stats(tokens, num_tokens = 5, verbose=True) :
    """
        Given a list of tokens, print number of tokens, number of unique tokens, 
        number of characters, lexical diversity, and num_tokens most common
        tokens. Return a list of 
    """

    
    
    num_tokens = len(tokens)
    
    num_unique_tokens = len(set(tokens))
    
    lexical_diversity = ld.ttr(tokens)
    
    num_characters = 0
    for word in tokens:
        num_characters+= len(word)
        
    common_tokens =  Counter(tokens)
    top_five_token = common_tokens .most_common(5)
    
    
    if verbose == True:        
        print(f"There are {num_tokens} tokens in the data.")
        print(f"There are {num_unique_tokens} unique tokens in the data.")
        print(f"There are {num_characters} characters in the data.")
        print(f"The lexical diversity is {lexical_diversity:.3f} in the data.")
    
        # print the five most common tokens
        
        print(f"The five most common tokens are : {top_five_token}")
       
          
        
    return([num_tokens, num_unique_tokens,
            lexical_diversity,
            num_characters,top_five_token])
    
    


def remove_stop(tokens) :
    
    # modify this function to remove stopwords
    return [w for w in tokens if w.lower() not in sw]
    
    

 
def remove_punctuation(text, punct_set=tw_punct) : 
    return("".join([ch for ch in text if ch not in punct_set]))



def tokenize(text) : 
    """ Splitting on whitespace rather than the book's tokenize function. That 
        function will drop tokens like '#hashtag' or '2A', which we need for Twitter. """
    
    # modify this function to return tokens
    return(text.split())


def prepare(text, pipeline) : 
    tokens = str(text)
    
    for transform in pipeline : 
        tokens = transform(tokens)
        
    return(tokens)

def join_tokens(tokens):
    
    text = " ".join(tokens)
    
    return(text)


In [43]:
my_pipeline = [str.lower, remove_punctuation, tokenize, remove_stop, join_tokens]

In [44]:
convention_db = sqlite3.connect(r"C:\Users\Mohammad\Downloads\2020_Conventions.db")

convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [45]:
query_results = convention_cur.execute(
                            '''
                            SELECT name FROM sqlite_schema
WHERE type='table'
ORDER BY name;
                            ''')

#Fetching 1st row from the table
# result = cursor.fetchone();
# print(result)

#Fetching all rows from the table
result = convention_cur.fetchall();
print(result)

[('conventions',)]


In [46]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

query_results = convention_cur.execute(
                            '''
                            SELECT text,party FROM conventions
                            ''')

for row in query_results :
    text,party=row
    clean_text=prepare(text, pipeline=my_pipeline)
    convention_data.append([clean_text,party])

    

Let's look at some random entries and see if they look right. 

In [47]:
random.choices(convention_data,k=10)

[['congratulations you’re citizens united states america behalf department homeland security honor call fellow americans mr president want commend dedication rule law restoring integrity immigration system thank hosting patriotic celebration white house today',
  'Republican'],
 ['i’m pleased announce vice president joe biden officially nominated democratic party candidate president united states',
  'Democratic'],
 ['every generation us fight believe it’s turn jack g 010203 proud saw demonstrations going across country',
  'Democratic'],
 ['joe biden selected kamala harris running mate', 'Democratic'],
 ['american history tells us that’s darkest moments we’ve made greatest progress found light dark moment believe we’re poised make great progress find light',
  'Democratic'],
 ['i’m reelected best yet come thank much took office middle east total chaos isis rampaging iran rise war afghanistan end sight',
  'Republican'],
 ['i’m rebecca friedrichs veteran california public school educat

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [48]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2391 as features in the model.


In [49]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    
    ret_dict = dict()
    
    for word in text.split():
        if word in fw:
            ret_dict [word]=True   
    return(ret_dict)

In [50]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [51]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [52]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [53]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.5


In [54]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     25.8 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                    isis = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

Among the 25 most frequently mentioned words, 23 are predominantly used by Republicans, while only 2 are associated with Democrats: 'climate' and 'votes'. It appears that the Republicans have a notable emphasis on China, though it's unclear whether this emphasis is positive or negative. Additionally, Republicans often mention terms such as 'liberal,' 'flag,' 'Abraham,' and 'freedoms,' which may suggest an appeal to patriotic sentiments.



## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [55]:
cong_db = sqlite3.connect(r"C:\Users\Mohammad\Downloads\congressional_data.db")
cong_cur = cong_db.cursor()

In [56]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [57]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.


for row in results :
    candidate, party,tweet_text=row
    clean_tweet_text=prepare(tweet_text, pipeline=my_pipeline)
    tweet_data.append([clean_tweet_text,party])


In [58]:
print("List Size:", len(tweet_data))

List Size: 664656


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [60]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)
tweet_data_sample

[['bearlier today spoke house floor abt protecting health care women praised ppmarmonte work central coast httpstcowqgtrzt7vv',
  'Democratic'],
 ['bgo tribe #rallytogether httpstco0nxutfl9l5', 'Democratic'],
 ['bapparently trump thinks easy students overwhelmed crushing burden debt pay student loans #trumpbudget httpstcockyqo5t0qh',
  'Democratic'],
 ['bwexe2x80x99re grateful first responders rescue personnel firefighters police volunteers working tirelessly keep people safe provide muchneeded help putting lives linennhttpstcoezpv0vmiz3',
  'Republican'],
 ['bletxe2x80x99s make even greater #kag xf0x9fx87xbaxf0x9fx87xb8 httpstcoy9qozd5l2z',
  'Republican'],
 ['bwe 1hr cavs tie series 22 im #allin216 repbarbaralee scared #roadtovictory',
  'Democratic'],
 ['bcongrats belliottsd new gig sd city hall glad continue servexe2x80xa6 httpstcofkvmw3cqdi',
  'Democratic'],
 ['bwe really close 3500 raised toward match right whoot thatxe2x80x99s 7000 nonmath majors room xf0x9fx98x82 help us get h

In [61]:
for tweet, party in tweet_data_sample :
    estimated_party = classifier.classify(conv_features(tweet,feature_words))
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: bearlier today spoke house floor abt protecting health care women praised ppmarmonte work central coast httpstcowqgtrzt7vv
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: bgo tribe #rallytogether httpstco0nxutfl9l5
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: bapparently trump thinks easy students overwhelmed crushing burden debt pay student loans #trumpbudget httpstcockyqo5t0qh
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: bwexe2x80x99re grateful first responders rescue personnel firefighters police volunteers working tirelessly keep people safe provide muchneeded help putting lives linennhttpstcoezpv0vmiz3
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: bletxe2x80x99s make even greater #kag xf0x9fx87xbaxf0x9fx87xb8 httpstcoy9qozd5l2z
Actual party is Republican and our classifer says

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [62]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   


    # get the estimated party
    estimated_party = classifier.classify(conv_features(tweet,feature_words))
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [63]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3580, 'Democratic': 698}),
             'Democratic': defaultdict(int,
                         {'Republican': 4712, 'Democratic': 1012})})

# Accuracy:

In [64]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in tweet_data]
accuracy=nltk.classify.accuracy(classifier, featuresets)
accuracy

0.46260922943597893

### Reflections.
Our model demonstrates a stronger predictive accuracy for Republicans, correctly identifying them at a rate of approximately 84% (3580 out of 4278), while its performance drops significantly for Democrats, with a correct identification rate of about 17.5% (1012 out of 5724). Consequently, the overall accuracy of this naive model falls below that of random guessing, with a score of 0.46 compared to the 0.50 benchmark.